# Projeto | Análise de Texto e Fontes Desestruturadas Web

## Feito por: Beni Kracochansky | Gabriel Kabbani | Rodrigo Mattar

A ideia do projeto é fazer um código que, dado a url do pdf do 10K de uma empresa listada americana, consiga baixar esse pdf e então fazer uma análise gráfica do faturamento e EBITDA da companhia em questão. 

Para testar o funcionamento do projeto, utilizamos o 10K (annual fillings para a SEC) da Apple.


In [2]:
import requests

### Extração dos dados:

In [3]:

data_url = "https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/b4266e40-1de6-4a34-9dfb-8632b8bd57e0.pdf" #link para o 10k da Apple

nome_saida = 'apple10k.pdf'

def baixa(pdf_url, nome_saida):
    response = requests.get(pdf_url) #pegando o pdf
    
    response.raise_for_status()  # vendo se está válido

    with open(nome_saida, 'wb') as output_file:
    
        output_file.write(response.content) #escrevendo arquvio de saida

baixa(data_url, nome_saida)